In [1]:
import numpy as np
import symforce
import os

# Symforce setup
symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")
symforce.set_epsilon_to_symbol()

from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

from sympy import linsolve

out_put_save_directory = os.getcwd()

In [2]:
t_i = sf.V3.symbolic("t_i")

t_f = sf.V3.symbolic("t_f")

s = sf.Symbol('s')
R = sf.Rot3.symbolic("R")
t_odo = sf.V3.symbolic("t_odo")
display(t_odo)

a_1 = sf.V3.symbolic("a_1")
a_2 = sf.V3.symbolic("a_2")
a_3 = sf.V3.symbolic("a_3")
a_4 = sf.V3.symbolic("a_4")

r_delta, r_TSTA, r_Break = sf.symbols("r_delta r_TSTA r_Break")
theta_delta, theta_TSTA, theta_Break = sf.symbols("theta_delta theta_TSTA theta_Break")
m_0, n_0, k_0, h_0 = sf.symbols("m_0 n_0 k_0 h_0")
l_0 = sf.V3.symbolic("")
l = sf.V3.symbolic("")
l_enc = sf.V3.symbolic("")

⎡t_odo0⎤
⎢      ⎥
⎢t_odo1⎥
⎢      ⎥
⎣t_odo2⎦

Odometry Error Model

In [3]:
odo_error = t_f - s*(R*t_odo) - t_i
display(odo_error)

⎡    ⎛       ⎛       2        2    ⎞                                          
⎢- s⋅⎝t_odo0⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + t_odo1⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + t_odo2
⎢                                                                             
⎢    ⎛                                       ⎛      2        2    ⎞           
⎢- s⋅⎝t_odo0⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + t_odo1⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + t_odo2⋅(
⎢                                                                             
⎢    ⎛                                                                        
⎣- s⋅⎝t_odo0⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + t_odo1⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + t_odo2⋅

                       ⎞             ⎤
⋅(2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⎠ + t_f0 - tᵢ₀⎥
                                     ⎥
                      ⎞              ⎥
-2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z)⎠ + t_f1 - tᵢ₁ ⎥
                                     ⎥
⎛      2        2    ⎞⎞              ⎥
⎝- 2⋅Rₓ  - 2⋅R_y  + 1⎠⎠ + t_f2 - tᵢ₂ ⎦

Kinematic Error Model Function

In [4]:
# inverse kinematic
l_0[0] = m_0 + n_0
l_0[1] = m_0 + 2*k_0
l_0[2] = n_0 + 2*h_0
m = sf.sqrt((t_f[0] - a_1[0])**2 + (t_f[1] - a_1[1])**2)
n = sf.sqrt((t_f[0] - a_2[0])**2 + (t_f[1] - a_2[1])**2)
k = sf.sqrt((t_f[0] - a_3[0])**2 + (t_f[1] - a_3[1])**2)
h = sf.sqrt((t_f[0] - a_4[0])**2 + (t_f[1] - a_4[1])**2)
l[0] = m + n
l[1] = m + 2*k
l[2] = n + 2*h
l_enc[0] = r_delta*theta_delta
l_enc[1] = r_TSTA*theta_TSTA
l_enc[2] = r_Break*theta_Break
error_ik = l - l_0 - l_enc
display(l)

⎡    _________________________________      _________________________________ 
⎢   ╱              2                2      ╱              2                2  
⎢ ╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)   + ╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)   
⎢                                                                             
⎢   _________________________________        _________________________________
⎢  ╱              2                2        ╱              2                2 
⎢╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)   + 2⋅╲╱  (-a₃₀ + t_f0)  + (-a₃₁ + t_f1)  
⎢                                                                             
⎢   _________________________________        _________________________________
⎢  ╱              2                2        ╱              2                2 
⎣╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)   + 2⋅╲╱  (-a₄₀ + t_f0)  + (-a₄₁ + t_f1)  

⎤
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎦

# Odometry Error Function

In [5]:
def odo_error_func(t_i: sf.V3,
                   t_f: sf.V3,
                   R: sf.Rot3,
                   s: sf.Symbol,
                   t_odo: sf.V3,
                   epsilon: sf.Scalar) ->sf.V3:
    display(odo_error)
    return odo_error

residual_func_codegen = codegen.Codegen.function(func=odo_error_func, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡    ⎛       ⎛       2        2    ⎞                                          
⎢- s⋅⎝t_odo0⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ + t_odo1⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + t_odo2
⎢                                                                             
⎢    ⎛                                       ⎛      2        2    ⎞           
⎢- s⋅⎝t_odo0⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) + t_odo1⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ + t_odo2⋅(
⎢                                                                             
⎢    ⎛                                                                        
⎣- s⋅⎝t_odo0⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) + t_odo1⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) + t_odo2⋅

                       ⎞             ⎤
⋅(2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⎠ + t_f0 - tᵢ₀⎥
                                     ⎥
                      ⎞              ⎥
-2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z)⎠ + t_f1 - tᵢ₁ ⎥
                                     ⎥
⎛      2        2    ⎞⎞              ⎥
⎝- 2⋅Rₓ  - 2⋅R_y  + 1⎠⎠ + t_f2 - tᵢ₂ ⎦

### Jacobian of Odometry Error Function with respect to $s$

In [6]:
def odo_error_func_wrt_s(t_i: sf.V3,
                         t_f: sf.V3,
                         R: sf.Rot3,
                         s: sf.Symbol,
                         t_odo: sf.V3,
                         epsilon: sf.Scalar) ->sf.V3:
    display(odo_error.diff(s))                     
    return odo_error.diff(s)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_s, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡         ⎛       2        2    ⎞                                             
⎢- t_odo0⋅⎝- 2⋅R_y  - 2⋅R_z  + 1⎠ - t_odo1⋅(-2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) - t_odo2⋅(2
⎢                                                                             
⎢                                         ⎛      2        2    ⎞              
⎢ -t_odo0⋅(2⋅R_w⋅R_z + 2⋅Rₓ⋅R_y) - t_odo1⋅⎝- 2⋅Rₓ  - 2⋅R_z  + 1⎠ - t_odo2⋅(-2⋅
⎢                                                                             
⎢                                                                          ⎛  
⎣ -t_odo0⋅(-2⋅R_w⋅R_y + 2⋅Rₓ⋅R_z) - t_odo1⋅(2⋅R_w⋅Rₓ + 2⋅R_y⋅R_z) - t_odo2⋅⎝- 

                    ⎤
⋅R_w⋅R_y + 2⋅Rₓ⋅R_z)⎥
                    ⎥
                    ⎥
R_w⋅Rₓ + 2⋅R_y⋅R_z) ⎥
                    ⎥
    2        2    ⎞ ⎥
2⋅Rₓ  - 2⋅R_y  + 1⎠ ⎦

### Jacobian of Odometry Error Function with respect to $R$

In [7]:
def odo_error_func_wrt_rot(t_i: sf.V3,
                             t_f: sf.V3,
                             R: sf.Rot3,
                             s: sf.Symbol,
                             t_odo: sf.V3,
                             epsilon: sf.Scalar):
    return odo_error.jacobian(R)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_rot, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Odometry Error Function with respect to $t_i$

In [8]:
def odo_error_func_wrt_pos_i(t_i: sf.V3,
                             t_f: sf.V3,
                             R: sf.Rot3,
                             s: sf.Symbol,
                             t_odo: sf.V3,
                             epsilon: sf.Scalar) ->sf.Matrix33:
    display(odo_error.jacobian(t_i))
    return odo_error.jacobian(t_i)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_pos_i, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡-1  0   0 ⎤
⎢          ⎥
⎢0   -1  0 ⎥
⎢          ⎥
⎣0   0   -1⎦

### Jacobian of Odometry Error Function with respect to $t_f$

In [9]:
def odo_error_func_wrt_pos_f(t_i: sf.V3,
                             t_f: sf.V3,
                             R: sf.Rot3,
                             s: sf.Symbol,
                             t_odo: sf.V3,
                             epsilon: sf.Scalar) ->sf.Matrix33:
    display(odo_error.jacobian(t_f))
    return odo_error.jacobian(t_f)

residual_func_codegen = codegen.Codegen.function(func=odo_error_func_wrt_pos_f, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

# Kinematic Error Function

In [10]:
def kin_error_func(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                   t_f: sf.V3,
                   m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                   r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                   theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                   epsilon: sf.Scalar) ->sf.V3:
    display(error_ik)
    return error_ik

residual_func_codegen = codegen.Codegen.function(func=kin_error_func, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡                              _________________________________      ________
⎢                             ╱              2                2      ╱        
⎢      -m₀ - n₀ - r_δ⋅θ_δ + ╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)   + ╲╱  (-a₂₀ +
⎢                                                                             
⎢                                 _________________________________        ___
⎢                                ╱              2                2        ╱   
⎢ -2⋅k₀ - m₀ - r_TSTA⋅θ_TSTA + ╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)   + 2⋅╲╱  (-
⎢                                                                             
⎢                                  _________________________________        __
⎢                                 ╱              2                2        ╱  
⎣-2⋅h₀ - n₀ - r_Break⋅θ_Break + ╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)   + 2⋅╲╱  (

_________________________      ⎤
      2                2       ⎥
 t_f0)  + (-a₂₁ + t_f1)        ⎥
                               

### Jacobian of Kinematic Error Function with respect to $a_1$

In [11]:
def kin_error_func_wrt_a_1(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.Matrix33:
    display(error_ik.jacobian(a_1))
    return error_ik.jacobian(a_1)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_1, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡          -(-a₁₀ + t_f0)                        -(-a₁₁ + t_f1)               
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎢╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)    ╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)     
⎢                                                                             
⎢          -(-a₁₀ + t_f0)                        -(-a₁₁ + t_f1)               
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎢╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)    ╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)     
⎢                                                                             
⎣                 0                                 

### Jacobian of Kinematic Error Function with respect to $a_2$

In [12]:
def kin_error_func_wrt_a_2(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.Matrix33:
    display(error_ik.jacobian(a_2))
    return error_ik.jacobian(a_2)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_2, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡          -(-a₂₀ + t_f0)                        -(-a₂₁ + t_f1)               
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎢╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)    ╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)     
⎢                                                                             
⎢                 0                                     0                    0
⎢                                                                             
⎢          -(-a₂₀ + t_f0)                        -(-a₂₁ + t_f1)               
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎣╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)    ╲╱  (-a₂₀ + t

### Jacobian of Kinematic Error Function with respect to $a_3$

In [13]:
def kin_error_func_wrt_a_3(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.Matrix33:
    display(error_ik.jacobian(a_3))
    return error_ik.jacobian(a_3)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_3, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡                 0                                     0                    0
⎢                                                                             
⎢         -2⋅(-a₃₀ + t_f0)                      -2⋅(-a₃₁ + t_f1)              
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎢╲╱  (-a₃₀ + t_f0)  + (-a₃₁ + t_f1)    ╲╱  (-a₃₀ + t_f0)  + (-a₃₁ + t_f1)     
⎢                                                                             
⎣                 0                                     0                    0

⎤
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎦

### Jacobian of Kinematic Error Function with respect to $a_4$

In [14]:
def kin_error_func_wrt_a_4(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.Matrix33:
    display(error_ik.jacobian(a_4))
    return error_ik.jacobian(a_4)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_a_4, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡                 0                                     0                    0
⎢                                                                             
⎢                 0                                     0                    0
⎢                                                                             
⎢         -2⋅(-a₄₀ + t_f0)                      -2⋅(-a₄₁ + t_f1)              
⎢────────────────────────────────────  ────────────────────────────────────  0
⎢   _________________________________     _________________________________   
⎢  ╱              2                2     ╱              2                2    
⎣╲╱  (-a₄₀ + t_f0)  + (-a₄₁ + t_f1)    ╲╱  (-a₄₀ + t_f0)  + (-a₄₁ + t_f1)     

⎤
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎦

### Jacobian of Kinematic Error Function with respect to $t_f$

In [15]:
def kin_error_func_wrt_pos_f(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                             t_f: sf.V3,
                             m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                             r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                             theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                             epsilon: sf.Scalar) ->sf.Matrix33:
    display(error_ik.jacobian(t_f))
    return error_ik.jacobian(t_f)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_pos_f, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡            -a₁₀ + t_f0                            -a₂₀ + t_f0               
⎢──────────────────────────────────── + ────────────────────────────────────  
⎢   _________________________________      _________________________________  
⎢  ╱              2                2      ╱              2                2   
⎢╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)     ╲╱  (-a₂₀ + t_f0)  + (-a₂₁ + t_f1)    
⎢                                                                             
⎢            -a₁₀ + t_f0                          2⋅(-a₃₀ + t_f0)             
⎢──────────────────────────────────── + ────────────────────────────────────  
⎢   _________________________________      _________________________________  
⎢  ╱              2                2      ╱              2                2   
⎢╲╱  (-a₁₀ + t_f0)  + (-a₁₁ + t_f1)     ╲╱  (-a₃₀ + t_f0)  + (-a₃₁ + t_f1)    
⎢                                                                             
⎢            -a₂₀ + t_f0                          2⋅

### Jacobian of Kinematic Error Function with respect to $m_0$

In [16]:
def kin_error_func_wrt_m_0(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.V3:
    display(error_ik.diff(m_0))
    return error_ik.diff(m_0)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_m_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

⎡-1⎤
⎢  ⎥
⎢-1⎥
⎢  ⎥
⎣0 ⎦

### Jacobian of Kinematic Error Function with respect to $n_0$

In [17]:
def kin_error_func_wrt_n_0(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.V3:
    return error_ik.diff(n_0)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_n_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $k_0$

In [18]:
def kin_error_func_wrt_k_0(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.V3:
    return error_ik.diff(k_0)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_k_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)

### Jacobian of Kinematic Error Function with respect to $h_0$

In [19]:
def kin_error_func_wrt_h_0(a_1: sf.V3, a_2: sf.V3, a_3: sf.V3, a_4: sf.V3,
                           t_f: sf.V3,
                           m_0: sf.Symbol, n_0: sf.Symbol, k_0: sf.Symbol, h_0: sf.Symbol,
                           r_delta: sf.Symbol, r_TSTA: sf.Symbol, r_Break: sf.Symbol,
                           theta_delta: sf.Symbol, theta_TSTA: sf.Symbol, theta_Break: sf.Symbol,
                           epsilon: sf.Scalar) ->sf.V3:
    return error_ik.diff(h_0)

residual_func_codegen = codegen.Codegen.function(func=kin_error_func_wrt_h_0, config=codegen.CppConfig(),)
residual_func_codegen_data = residual_func_codegen.generate_function(output_dir=out_put_save_directory)